# Analysis of the 2nd 2024 Barometer of Catalonia (pt1)
# Preprocessing the survey

Quantitative survey conducted by the Centre d'Estudis d'Opinió (CEO) between June 10 and July 8, 2024, based on a sample of 2,000 individuals. The target population is the Catalan population, and the survey gathers public opinion on politics, economy, media, and electoral behavior.

The data used for the analysis can be found on the following url: https://ceo.gencat.cat/ca/barometre/detall/index.html?id=9368

In this Jupyter Notebook I will preprocess the survey dataset and get a cleaned one which I can use for exploratory data analysis and training ML models.

_________________________________________________

Libraries used for the preprocessing:

In [1]:
import pandas as pd
import numpy as np

## 00. Importing the data

Let's start importing the file:

In [2]:
bar2024 = pd.read_excel('2nBar2024.xlsx')
bar2024

,PONDERA,ORDRE_REVISADA,REO,METODOLOGIA,BOP_NUM,ANY,MES,DIA,DATA_INI,HORA_INI,...,INT_PARLAMENT_VOT_R,REC_PARLAMENT_UE_VOT_R,REC_PARLAMENT_UE_VOT_CENS_R,REC_PARLAMENT_VOT_R,REC_PARLAMENT_VOT_CENS_R,CIRCUIT_1101_1,CIRCUIT_1101_2,EGP10,EGP6,INDEX_BENESTAR
0,1,1,1101,Presencial,Juliol 24 -1101,2024,Juny,10,2024-10-06 00:00:00,,...,Junts per Catalunya,,No va votar,No ho sap,No ho sap,VAL_GOV_CAT després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,,,3
1,1,2,1101,Presencial,Juliol 24 -1101,2024,Juny,10,2024-10-06 00:00:00,,...,Comuns Sumar,Comuns Sumar,Comuns Sumar,Comuns Sumar,Comuns Sumar,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim catalanisme i 10=màxim espanyolisme,I Classe de Serveis alta,I+II Classe de serveis,5
2,1,3,1101,Presencial,Juliol 24 -1101,2024,Juny,10,2024-10-06 00:00:00,,...,En blanc,,No ho sap,En blanc,En blanc,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,VIIb Treballadors/es del sector primari,IVc+VIIb Treballadors/es i empresaris/àries se...,4
3,1,4,1101,Presencial,Juliol 24 -1101,2024,Juny,10,2024-10-06 00:00:00,,...,Altres,Nul,Nul,PSC/PSOE,PSC/PSOE,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,IVb Autònoms/es sense treballadors,IVab Petita burgesia,4
4,1,5,1101,Presencial,Juliol 24 -1101,2024,Juny,10,2024-10-06 00:00:00,,...,PSC/PSOE,PSC/PSOE,PSC/PSOE,PSC/PSOE,PSC/PSOE,VAL_GOV_CAT després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,I Classe de Serveis alta,I+II Classe de serveis,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,1996,1101,Presencial,Juliol 24 -1101,2024,Juliol,5,2024-05-07 00:00:00,,...,CUP,ERC,ERC,CUP,CUP,VAL_GOV_CAT després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,VIIa Feines manuals semi i no-qualificades,VIIa Treballadors/es manuals semi i no-qualifi...,4
1996,1,1997,1101,Presencial,Juliol 24 -1101,2024,Juliol,8,2024-08-07 00:00:00,,...,ERC,,No va votar,,No va votar,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim catalanisme i 10=màxim espanyolisme,VIIa Feines manuals semi i no-qualificades,VIIa Treballadors/es manuals semi i no-qualifi...,3
1997,1,1998,1101,Presencial,Juliol 24 -1101,2024,Juliol,8,2024-08-07 00:00:00,,...,Vox,,No va votar,,No va votar,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim catalanisme i 10=màxim espanyolisme,VIIa Feines manuals semi i no-qualificades,VIIa Treballadors/es manuals semi i no-qualifi...,3
1998,1,1999,1101,Presencial,Juliol 24 -1101,2024,Juliol,8,2024-08-07 00:00:00,,...,Vox,VOX,VOX,Vox,Vox,VAL_GOV_ESP després d’INTERES_POL_PUBLICS; VAL...,0=màxim espanyolisme i 10=màxim catalanisme,IVa Empresaris/àries excepte classe I,IVab Petita burgesia,4


The file ```Variables_analisi``` contains the variables that will be used for the analysis and information about them.

Column ```Grup``` indicates if the variable contains demographic or political information about the individual.\
Column ```Tipus``` indicates if the variable is numerical or cathegorical.\
Column ```Bloc_num``` indicates if the variable shares an information pattern. For example, questions related to the simpathy for political parties. 

In [3]:
var_an = pd.read_excel('Variables_analisi.xlsx')
var_an

,Variables,Grup,Tipus,Bloc_num
0,PROVINCIA,d,c,NaN
1,MUNICIPI,d,c,NaN
2,COMARCA,d,c,NaN
3,CIUTADANIA,d,c,NaN
4,SEXE,d,c,NaN
...,...,...,...,...
76,SENTIMENT_PERTINENCA,p,c,NaN
77,CLASSE_SOCIAL_SUBJECTIVA_1_7,d,c,NaN
78,PERSONES_LLAR,d,n,NaN
79,ORIENTACIO_SEXUAL,d,c,NaN


Let's see the number of variables that contain political ```"p"``` and demographic ```"d"``` information:

In [4]:
var_an["Grup"].value_counts()

Grup
p    55
d    26
Name: count, dtype: int64

I will get only the columns which are useful for the analysis. 

In [5]:
df1 = bar2024[ list( var_an["Variables"] ) ]

_______________________

## 01. Cleaning the data

### 01.01 Missing data

Let's see how the missings are distributed:

In [6]:
df1.isna().sum()

PROVINCIA                       0
MUNICIPI                        0
COMARCA                         0
CIUTADANIA                      0
SEXE                            0
                               ..
SENTIMENT_PERTINENCA            0
CLASSE_SOCIAL_SUBJECTIVA_1_7    0
PERSONES_LLAR                   0
ORIENTACIO_SEXUAL               0
INGRESSOS_1_15                  0
Length: 81, dtype: int64

In [7]:
miss_group = (df1 == " ").sum()
miss_taula = pd.DataFrame(
    {
    'variable': miss_group.index,
    'n_miss': miss_group.values
    }
)

There are no missings. If a data point is not encoded is saved as "Not answered".

In [8]:
miss_taula[miss_taula["n_miss"]>0]

,variable,n_miss
1,MUNICIPI,360
7,PROBLEMA_REDUIDA,136


It's better to fill the data with a ```No contesta```.

In [9]:
df1.loc[df1["MUNICIPI"]==" ", "MUNICIPI"] = "No contesta"
df1.loc[df1["PROBLEMA_REDUIDA"]==" ", "PROBLEMA_REDUIDA"] = "No contesta"

### 01.02 Encoding numerical data

I have to encode properly different numerical variables. Some numerical variables share a codification pattern, so I will code the 4 different groups. I will create 4 functions that encode the variable for each one of the 4 groups.

The numerical variables used for the analysis are the following ones with his group.

In [10]:
print("Number of numerical variables to encode:", var_an[var_an["Tipus"]=="n"].shape[0] )

var_an[var_an["Tipus"]=="n"]

Number of numerical variables to encode: 36


,Variables,Grup,Tipus,Bloc_num
5,EDAT,d,n,NaN
23,VAL_GOV_CAT,p,n,1.0
24,VAL_GOV_ESP,p,n,1.0
33,IDEOL_0_10,p,n,2.0
34,IDEOL_0_10_PP,p,n,2.0
35,IDEOL_0_10_ERC,p,n,2.0
36,IDEOL_0_10_PSC,p,n,2.0
37,IDEOL_0_10_CUP,p,n,2.0
38,IDEOL_0_10_JXCAT,p,n,2.0
39,IDEOL_0_10_CEC,p,n,2.0


Let's create a copy of the dataframe in which I will apply the codification

In [11]:
df2 = df1.copy()

- **Group 1:** Valoration of Catalan and Spanish Government

In [12]:
vars_g1 = list( var_an[var_an["Bloc_num"]==1]["Variables"] )
vars_g1

['VAL_GOV_CAT', 'VAL_GOV_ESP']

I apply the codification and I change the variable type to integer.

In [13]:
def codificar_G1(variable):
    df2.loc[ df2[variable] == "0 Molt dolenta", variable] = 0
    df2.loc[ df2[variable] == "10 Molt bona", variable] = 10
    df2.loc[ df2[variable] == "10  Molt bona", variable] = 10
    df2.loc[ df2[variable] == "0 Molt dolenta", variable] = 0
    df2.loc[ df2[variable] == "No ho sap", variable] = np.nan
    df2.loc[ df2[variable] == "No contesta", variable] = np.nan
    df2[variable].astype('Int64')
    
    
for var in vars_g1:
    codificar_G1(var)

- **Group 2:** Left-wing ideology and ranking the ideology of other political parties

In [14]:
vars_g2 = list( var_an[var_an["Bloc_num"]==2]["Variables"] )
vars_g2

['IDEOL_0_10',
 'IDEOL_0_10_PP',
 'IDEOL_0_10_ERC',
 'IDEOL_0_10_PSC',
 'IDEOL_0_10_CUP',
 'IDEOL_0_10_JXCAT',
 'IDEOL_0_10_CEC',
 'IDEOL_0_10_VOX',
 'IDEOL_0_10_ALIANCA']

I apply the codification and I change the variable type to integer.

In [15]:
def codificar_G2(variable):
    df2.loc[ df2[variable] == "0 Extrema esquerra", variable] = 0
    df2.loc[ df2[variable] == "10 Extrema dreta", variable] = 10
    df2.loc[ df2[variable] == "No ho sap", variable] = np.nan
    df2.loc[ df2[variable] == "No contesta", variable] = np.nan
    df2[variable].astype('Int64')

    
for var in vars_g2:
    codificar_G2(var)

- **Group 3:** Catalan independence - Spanish unionism ideology and ranking the ideology of other political parties

In [16]:
vars_g3 = list( var_an[var_an["Bloc_num"]==3]["Variables"] )
vars_g3

['ESP_CAT_0_10',
 'ESP_CAT_0_10_PP',
 'ESP_CAT_0_10_ERC',
 'ESP_CAT_0_10_PSC',
 'ESP_CAT_0_10_CUP',
 'ESP_CAT_0_10_JXCAT',
 'ESP_CAT_0_10_CEC',
 'ESP_CAT_0_10_VOX',
 'ESP_CAT_0_10_ALIANCA']

I apply the codification and I change the variable type to integer.

In [17]:
def codificar_G3(variable):
    df2.loc[ df2[variable] == "0 Màxim espanyolisme", variable] = 0
    df2.loc[ df2[variable] == "10 Màxim catalanisme", variable] = 10
    df2.loc[ df2[variable] == "No ho sap", variable] = np.nan
    df2.loc[ df2[variable] == "No contesta", variable] = np.nan
    df2[variable].astype('Int64')

    
for var in vars_g3:
    codificar_G3(var)

- **Group 4:** Simpathy for groups and political parties

In [18]:
vars_g4 = list( var_an[var_an["Bloc_num"]==4]["Variables"] )
vars_g4

['SIMPATIA_INDEPENDENTISTES_0_10',
 'SIMPATIA_UNIONISTES_0_10',
 'SIMPATIA_ESQUERRES_0_10',
 'SIMPATIA_DRETES_0_10',
 'SIMPATIA_FEMINISTES_0_10',
 'SIMPATIA_CONTRARIS_FEMINISME_0_10',
 'SIMPATIA_VOTANTS_PP_0_10',
 'SIMPATIA_VOTANTS_PSC_0_10',
 'SIMPATIA_VOTANTS_ERC_0_10',
 'SIMPATIA_VOTANTS_CUP_0_10',
 'SIMPATIA_VOTANTS_JXCAT_0_10',
 'SIMPATIA_VOTANTS_CEC_0_10',
 'SIMPATIA_VOTANTS_VOX_0_10',
 'SIMPATIA_VOTANTS_ALIANCA_0_10']

I apply the codification and I change the variable type to integer.

In [19]:
def codificar_G4(variable):
    df2.loc[ df2[variable] == "0 Molta antipatia", variable] = 0
    df2.loc[ df2[variable] == "10 Molta simpatia", variable] = 10
    df2.loc[ df2[variable] == "No ho sap", variable] = np.nan
    df2.loc[ df2[variable] == "No contesta", variable] = np.nan
    df2[variable].astype('Int64')

    
for var in vars_g4:
    codificar_G4(var)

Finally I encode one last variable:

In [20]:
df2.loc[ df2["PERSONES_LLAR"]=="Una", "PERSONES_LLAR"] = 1
df2.loc[ df2["PERSONES_LLAR"]=="Dos", "PERSONES_LLAR"] = 2
df2.loc[ df2["PERSONES_LLAR"]=="Tres", "PERSONES_LLAR"] = 3
df2.loc[ df2["PERSONES_LLAR"]=="Quatre", "PERSONES_LLAR"] = 4
df2.loc[ df2["PERSONES_LLAR"]=="Cinc", "PERSONES_LLAR"] = 5
df2.loc[ df2["PERSONES_LLAR"]=="sis", "PERSONES_LLAR"] = 6
df2.loc[ df2["PERSONES_LLAR"]=="Set", "PERSONES_LLAR"] = 7
df2.loc[ df2["PERSONES_LLAR"]=="Vuit", "PERSONES_LLAR"] = 8
df2.loc[ df2["PERSONES_LLAR"]=="Nou", "PERSONES_LLAR"] = 9
df2.loc[ df2["PERSONES_LLAR"]=="Deu", "PERSONES_LLAR"] = 10
df2.loc[ df2["PERSONES_LLAR"]=="Onze", "PERSONES_LLAR"] = 11
df2.loc[ df2["PERSONES_LLAR"]=="No contesta", "PERSONES_LLAR"] = np.nan
df2["PERSONES_LLAR"] = df2["PERSONES_LLAR"].astype('float')
df2.loc[ df2["PERSONES_LLAR"].isna()==True, "PERSONES_LLAR"] = df2["PERSONES_LLAR"].mean()

Let's see how the numerical variables look after the encoding:

In [21]:
df2[list(var_an[var_an["Tipus"]=="n"]["Variables"])].head(20)

,EDAT,VAL_GOV_CAT,VAL_GOV_ESP,IDEOL_0_10,IDEOL_0_10_PP,IDEOL_0_10_ERC,IDEOL_0_10_PSC,IDEOL_0_10_CUP,IDEOL_0_10_JXCAT,IDEOL_0_10_CEC,...,SIMPATIA_CONTRARIS_FEMINISME_0_10,SIMPATIA_VOTANTS_PP_0_10,SIMPATIA_VOTANTS_PSC_0_10,SIMPATIA_VOTANTS_ERC_0_10,SIMPATIA_VOTANTS_CUP_0_10,SIMPATIA_VOTANTS_JXCAT_0_10,SIMPATIA_VOTANTS_CEC_0_10,SIMPATIA_VOTANTS_VOX_0_10,SIMPATIA_VOTANTS_ALIANCA_0_10,PERSONES_LLAR
0,19,4,6,6,9,3,6,4,3,5,...,0,4,5,6,6,6,6,0,6,3.0
1,59,2,8,2,8,3,5,2,8,2,...,2,1,7,4,6,1,10,0,1,2.0
2,68,3,5,3,10,8,4,2,10,3,...,0,0,5,2,2,0,2,0,0,2.0
3,51,0,4,0,10,5,6,0,10,4,...,0,0,5,5,8,0,8,0,0,1.0
4,70,3,8,3,9,7,5,1,9,1,...,0,0,10,5,0,0,7,0,0,2.0
5,36,6,5,2,10,3,4,NaN,4,4,...,3,3,7,8,8,8,5,0,8,4.0
6,53,5,3,4,8,4,5,3,7,4,...,3,3,5,8,7,8,7,0,1,4.0
7,71,NaN,3,5,5,5,0,10,10,0,...,5,5,5,5,5,5,5,5,NaN,3.0
8,38,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,10,10,NaN,NaN,NaN,NaN,0,NaN,7.0
9,62,4,5,4,5,5,4,NaN,3,5,...,4,7,6,5,4,4,4,7,6,1.0


_____________________

## 02. Export the Data

Let's save the dataframe with the encoded variables as an excel

In [22]:
df2.to_excel('cleaned_df.xlsx',index=False)